In [ ]:
import os
from PIL import Image, ImageStat
import numpy as np
import requests
from io import BytesIO
import math

def enhance_image(image_path):
    # read image
    original_image = Image.open(image_path)
    
    # calculate brightness index
    brightness_index = ImageStat.Stat(original_image).mean[0]

    # check if image is bright enough
    if np.mean(original_image) > 150:
        print("Skipping image", image_path, "because it is already bright")
        return original_image

    # enhance image by 20% if it is too dark
    print("Enhancing image", image_path)
    width, height = original_image.size
    pixels = original_image.load()
    if original_image.mode == 'RGBA':
        for x in range(width):
            for y in range(height):
                r, g, b, a = pixels[x, y]
                new_r = min(255, int(r * 1.4))
                new_g = min(255, int(g * 1.4))
                new_b = min(255, int(b * 1.4))
                pixels[x, y] = (new_r, new_g, new_b, a)
    else:
        for x in range(width):
            for y in range(height):
                r, g, b = pixels[x, y]
                new_r = min(255, int(r * 1.4))
                new_g = min(255, int(g * 1.4))
                new_b = min(255, int(b * 1.4))
                pixels[x, y] = (new_r, new_g, new_b)
    enhanced_image = original_image

    return enhanced_image

def calculate_mse(original_image, enhanced_image):
    # calculate mean squared error (MSE)
    mse = np.mean((np.array(original_image) - np.array(enhanced_image)) ** 2)
    return mse

def calculate_psnr(original_image, enhanced_image):
    # calculate peak signal-to-noise ratio (PSNR) in decibels (dB)
    mse = calculate_mse(original_image, enhanced_image)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr
# input and output directories
# input and output directories
input_dir = 'input'
output_dir = 'output'

# create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# initialize lists to store MSE and PSNR values for each image
mse_list = []
psnr_list = []
brightness_list = []

# get list of input filenames
input_filenames = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# iterate over input images
for i, filename in enumerate(input_filenames):
    # enhance image
    enhanced_image = enhance_image(os.path.join(input_dir, filename))

    # calculate MSE and PSNR scores
    original_image = Image.open(os.path.join(input_dir, filename))
    mse = calculate_mse(original_image, enhanced_image)
    psnr = calculate_psnr(original_image, enhanced_image)
    brightness = ImageStat.Stat(original_image).mean[0]
    
    # append MSE and PSNR values to respective lists
    mse_list.append(mse)
    psnr_list.append(psnr)
    brightness_list.append(brightness)
    
    
    # save enhanced image
    output_path = os.path.join(output_dir, filename)
    if enhanced_image.mode == 'RGBA':
        enhanced_image = enhanced_image.convert('RGB')
    enhanced_image.save(output_path, format='JPEG')

    # print scores
    print("Image:", filename)
    print("MSE:", mse)
    print("PSNR:", psnr)
    print("brightness:", brightness)
    print("------------------------------------")

# calculate average MSE and PSNR values for all images
avg_mse = sum(mse_list) / len(mse_list)
avg_psnr = sum(psnr_list) / len(psnr_list)
avg_brightness = sum(brightness_list) / len(brightness_list)

# print average MSE and PSNR values
print("Average MSE:", avg_mse)
print("Average PSNR:", avg_psnr)
print("Average brightness:", avg_brightness)
